## RecursiveCharacterTextSplitter
`RecursiveCharacterTextSplitter`는 문서를 분할할 때 **설정된 문자 목록을 순서대로 사용하여 가장 적절한 크기로 나누는 방식**입니다.

- 우선적으로 `"\n\n"`(문단 단위)로 분할을 시도하고, 이후 `"\n"`(줄 단위) → `" "`(단어 단위) → `""`(문자 단위) 순으로 진행됨.
- `RecursiveCharacterTextSplitter`는 문단 → 문장 → 단어 순으로 분할을 시도하여 문장이 끊기지 않도록 **가능한 한 문맥을 유지한 상태로 텍스트를 나눌 수 있음**.
- 문맥 유지를 위한 설정
    - `chunk_overlap`은 chunk_size의 20~30% 정도로 설정함
    - **`separators`를 활용해 의미 단위로 끊기도록 조정** (`["\n\n", ".", "!", "?", " ", ""]`)
- 연구 논문, 기술 문서처럼 **구조가 명확하지 않은 긴 텍스트를 효과적으로 나눌 때 유용함**
- 계층적 분할로 **의미 단위를 최대한 보존**하면서 적절한 크기의 청크를 만들 수 있습니다.

In [1]:
# 간단 예제
from langchain.text_splitter import RecursiveCharacterTextSplitter

text = """RAG는 검색과 생성 단계를 포함하는 모델입니다.

이 모델은 검색 기반의 텍스트 생성 기능을 제공합니다.
특히, 최신 데이터를 반영하는 데 강력한 기능을 가지고 있습니다.

Transformer 모델을 기반으로 실시간 정보를 활용할 수 있으며, 기존의 단순한 생성 모델보다 더 정확한 답변을 제공합니다."""

# 의미 단위(문장, 단락)로 나누되, chunk_size 50 제한
splitter = RecursiveCharacterTextSplitter(chunk_size=80, chunk_overlap=20, separators=["\n\n", ".", "!", "?", " ", ""])
chunks = splitter.split_text(text)

for i, chunk in enumerate(chunks):
    print(f" Chunk {i+1}: {chunk}\n")

 Chunk 1: RAG는 검색과 생성 단계를 포함하는 모델입니다.

 Chunk 2: 이 모델은 검색 기반의 텍스트 생성 기능을 제공합니다.
특히, 최신 데이터를 반영하는 데 강력한 기능을 가지고 있습니다.

 Chunk 3: Transformer 모델을 기반으로 실시간 정보를 활용할 수 있으며, 기존의 단순한 생성 모델보다 더 정확한 답변을 제공합니다.



In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 예제 텍스트
text = """RAG는 검색과 생성 단계를 포함하는 모델입니다.

이 모델은 검색 기반의 텍스트 생성 기능을 제공합니다.
특히, 최신 데이터를 반영하는 데 강력한 기능을 가지고 있습니다.

Transformer 모델을 기반으로 실시간 정보를 활용할 수 있으며, 기존의 단순한 생성 모델보다 더 정확한 답변을 제공합니다.

RAG의 핵심은 검색과 생성의 결합입니다! 먼저 관련 문서를 찾고, 그 정보를 바탕으로 답변을 만듭니다."""

print("원본 텍스트:")
print("-" * 50)
print(text)
print(f"\n텍스트 길이: {len(text)}자")

# ===========================================
# Recursive vs Character 비교
# ===========================================

print("\n" + "="*60)
print("RecursiveCharacterTextSplitter vs CharacterTextSplitter 비교")
print("="*60)

# 1. RecursiveCharacterTextSplitter (추천)
print("\n1. RecursiveCharacterTextSplitter (계층적 분할):")
print("-" * 45)
recursive_splitter = RecursiveCharacterTextSplitter(
    chunk_size=80,
    chunk_overlap=20,
    separators=["\n\n", "\n", ".", "!", "?", " ", ""]  # 우선순위 순서
)
recursive_chunks = recursive_splitter.split_text(text)

for i, chunk in enumerate(recursive_chunks):
    print(f"Chunk {i+1}: '{chunk.strip()}'")
    print(f"길이: {len(chunk)}자")
    print()

# 2. CharacterTextSplitter (비교용)
print("2. CharacterTextSplitter (단순 분할):")
print("-" * 35)
from langchain.text_splitter import CharacterTextSplitter
simple_splitter = CharacterTextSplitter(
    chunk_size=80,
    chunk_overlap=20,
    separator="."  # 하나의 구분자만 사용
)
simple_chunks = simple_splitter.split_text(text)

for i, chunk in enumerate(simple_chunks):
    print(f"Chunk {i+1}: '{chunk.strip()}'")
    print(f"길이: {len(chunk)}자")
    print()

# ===========================================
# separators 우선순위 테스트
# ===========================================

print("="*60)
print("separators 우선순위 동작 확인")
print("="*60)

test_text = """첫 번째 문단입니다.

두 번째 문단입니다.
이 문단은 여러 문장으로 구성됩니다! 정말 흥미롭죠?

세 번째 문단입니다."""

print("테스트 텍스트:")
print(repr(test_text))  # 줄바꿈 문자까지 보이도록

# 다양한 separators 설정 테스트
separators_configs = [
    (["\n\n", "\n", ".", " "], "문단 우선"),
    (["\n", ".", " "], "줄바꿈 우선"),
    ([".", "!", "?", " "], "문장 우선"),
    ([" "], "단어 단위")
]

for separators, description in separators_configs:
    print(f"\n{description} separators={separators}:")
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=40,
        chunk_overlap=10,
        separators=separators
    )
    chunks = splitter.split_text(test_text)
    
    for i, chunk in enumerate(chunks, 1):
        print(f"  Chunk {i}: '{chunk.strip()}'")

# ===========================================
# chunk_size별 결과 비교
# ===========================================

print("\n" + "="*60)
print("chunk_size별 분할 결과 비교")
print("="*60)

chunk_sizes = [50, 100, 150]

for size in chunk_sizes:
    print(f"\nchunk_size={size}:")
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=size,
        chunk_overlap=20,
        separators=["\n\n", "\n", ".", " "]
    )
    chunks = splitter.split_text(text)
    
    print(f"총 {len(chunks)}개 청크 생성")
    avg_length = sum(len(chunk) for chunk in chunks) / len(chunks)
    print(f"평균 청크 길이: {avg_length:.1f}자")
    
    for i, chunk in enumerate(chunks, 1):
        print(f"  Chunk {i}: '{chunk.strip()[:30]}...' (길이: {len(chunk)}자)")

# ===========================================
# chunk_overlap 효과 확인
# ===========================================

print("\n" + "="*60)
print("chunk_overlap 효과 확인")
print("="*60)

overlap_values = [0, 10, 30]

for overlap in overlap_values:
    print(f"\nchunk_overlap={overlap}:")
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=80,
        chunk_overlap=overlap,
        separators=["\n\n", ".", " "]
    )
    chunks = splitter.split_text(text)
    
    print(f"총 {len(chunks)}개 청크 생성")
    for i, chunk in enumerate(chunks, 1):
        print(f"  Chunk {i}: '{chunk.strip()}'")
        
        # 중복 부분 확인
        if i > 1 and overlap > 0:
            prev_chunk = chunks[i-2].strip()
            curr_chunk = chunk.strip()
            # 간단한 중복 확인 (마지막 10자와 첫 10자 비교)
            if len(prev_chunk) >= 10 and len(curr_chunk) >= 10:
                prev_end = prev_chunk[-10:]
                curr_start = curr_chunk[:10]
                if any(word in curr_start for word in prev_end.split() if len(word) > 2):
                    print(f"    중복 감지: 이전 청크와 겹치는 부분 있음")

# ===========================================
# 활용 가이드
# ===========================================

print("\n" + "="*60)
print("실무 활용 가이드")
print("="*60)

print("""
RecursiveCharacterTextSplitter 사용 가이드:

1. 기본 설정 (일반적 문서):
   chunk_size=1000, chunk_overlap=200
   separators=["\n\n", "\n", ".", " "]

2. 한국어 문서 최적화:
   chunk_size=500-1000, chunk_overlap=100-200
   separators=["\n\n", "\n", ".", "。", " "]

3. 코드 문서:
   separators=["\n\n", "\n", "\t", " "]

4. 대화/채팅 로그:
   separators=["\n\n", "\n", ":", " "]

장점:
- 의미 단위로 자연스러운 분할
- 계층적 구분자로 최적화된 분할점 찾기
- 텍스트 특성에 맞는 유연한 설정

주의사항:
- chunk_size는 LLM 토큰 제한 고려
- chunk_overlap은 맥락 보존과 비용의 균형
- separators 순서가 분할 품질 결정
""")

print("\n프로그램 완료")

원본 텍스트:
--------------------------------------------------
RAG는 검색과 생성 단계를 포함하는 모델입니다.

이 모델은 검색 기반의 텍스트 생성 기능을 제공합니다.
특히, 최신 데이터를 반영하는 데 강력한 기능을 가지고 있습니다.

Transformer 모델을 기반으로 실시간 정보를 활용할 수 있으며, 기존의 단순한 생성 모델보다 더 정확한 답변을 제공합니다.

RAG의 핵심은 검색과 생성의 결합입니다! 먼저 관련 문서를 찾고, 그 정보를 바탕으로 답변을 만듭니다.

텍스트 길이: 230자

RecursiveCharacterTextSplitter vs CharacterTextSplitter 비교

1. RecursiveCharacterTextSplitter (계층적 분할):
---------------------------------------------
Chunk 1: 'RAG는 검색과 생성 단계를 포함하는 모델입니다.'
길이: 27자

Chunk 2: '이 모델은 검색 기반의 텍스트 생성 기능을 제공합니다.
특히, 최신 데이터를 반영하는 데 강력한 기능을 가지고 있습니다.'
길이: 67자

Chunk 3: 'Transformer 모델을 기반으로 실시간 정보를 활용할 수 있으며, 기존의 단순한 생성 모델보다 더 정확한 답변을 제공합니다.'
길이: 72자

Chunk 4: 'RAG의 핵심은 검색과 생성의 결합입니다! 먼저 관련 문서를 찾고, 그 정보를 바탕으로 답변을 만듭니다.'
길이: 58자

2. CharacterTextSplitter (단순 분할):
-----------------------------------
Chunk 1: 'RAG는 검색과 생성 단계를 포함하는 모델입니다.

이 모델은 검색 기반의 텍스트 생성 기능을 제공합니다'
길이: 58자

Chunk 2: '특히, 최신 데이터를 반영하는 데 강력한 기능을 가지고 있습니다'
길이: 35자

Chunk 3: 'Transformer 모델을 기반으로 실